### Forecast PFAS Data

Goal: The goal of this project is to train a model to forecast future levels of PFAS contamination
in different regions using historical PFAS data. The PFAS data collected by the EPA spans
multiple years and regions, and students will use this data to predict future PFAS levels.
Specifically, students will train models using data from 2001 to 2023 to forecast PFAS levels in
2024. The objective is to explore how well models can predict future growth or changes in PFAS
levels based on historical trends.

Data url: https://www.epa.gov/dwucmr/occurrence-data-unregulated-contaminant-monitoring-rule#4

### Import Data

In [1]:
import os
import sys

# Determine root directory based on OS
if sys.platform == "win32":  # Windows
    root_dir = r"C:\Users\kdubf\OneDrive\projects\cs_6463_final\ucmr5"
else:  # macOS
    root_dir = "/Users/kb/Library/CloudStorage/OneDrive-Personal/projects/cs_6463_final/ucmr5"

if not os.path.exists(root_dir):
    raise Exception(f"Directory not found: {root_dir}")

print(f"Using directory: {root_dir}")

Using directory: /Users/kb/Library/CloudStorage/OneDrive-Personal/projects/cs_6463_final/ucmr5


In [2]:
import pandas as pd

def read_txt_to_df(file_path):
    """Helper function to read txt file into dataframe with consistent settings"""
    # Read the header to get column names
    headers = pd.read_csv(file_path, sep='\t', encoding="ISO-8859-1", nrows=0).columns
    
    # Create a dictionary to specify all columns as string type
    dtype_dict = {col: str for col in headers}
    
    # Read the full file with string dtypes
    return pd.read_csv(
        file_path,
        sep='\t',
        encoding="ISO-8859-1",
        dtype=dtype_dict,
        keep_default_na=False,  # Prevent pandas from converting certain strings to NaN
        na_filter=False  # Don't interpret any values as NA/NaN
    )

# Read each file into its own dataframe
try:
    print("Reading UCMR5_AddtlDataElem.txt...")
    df_addtl = read_txt_to_df(os.path.join(root_dir, 'UCMR5_AddtlDataElem.txt'))
    print(f"Shape: {df_addtl.shape}")
    print("-" * 50)
except Exception as e:
    print(f"Error reading UCMR5_AddtlDataElem.txt: {str(e)}")
    df_addtl = None

try:
    print("Reading UCMR5_All.txt...")
    df_all = read_txt_to_df(os.path.join(root_dir, 'UCMR5_All.txt'))
    print(f"Shape: {df_all.shape}")
    print("-" * 50)
except Exception as e:
    print(f"Error reading UCMR5_All.txt: {str(e)}")
    df_all = None

try:
    print("Reading UCMR5_ZIPCodes.txt...")
    df_zip = read_txt_to_df(os.path.join(root_dir, 'UCMR5_ZIPCodes.txt'))
    print(f"Shape: {df_zip.shape}")
    print("-" * 50)
except Exception as e:
    print(f"Error reading UCMR5_ZIPCodes.txt: {str(e)}")
    df_zip = None



Reading UCMR5_AddtlDataElem.txt...
Shape: (298135, 7)
--------------------------------------------------
Reading UCMR5_All.txt...
Shape: (948284, 24)
--------------------------------------------------
Reading UCMR5_ZIPCodes.txt...
Shape: (30304, 2)
--------------------------------------------------


## Modify All Dataset

### Convert CollectionDate to datetime and MRL & AnalyticalResultValue to numeric values.

In [ ]:
# Filter out lithium records from df_all. Lithium is neither PFAS nor does it correlate to any PFAS.
df_all = df_all[df_all['Contaminant'] != 'lithium']

# Convert CollectionDate to datetime
df_all['CollectionDate'] = pd.to_datetime(df_all['CollectionDate'])

# Create a new column CollectionDate_FOM that is the first of the month and year of CollectionDate
df_all['CollectionDate_FOM'] = df_all['CollectionDate'].dt.to_period('M').dt.to_timestamp()

# Create a new column that is the 4-digit year and 2-digit month of CollectionDate
df_all['CollectionDate_YYYYMM'] = df_all['CollectionDate'].dt.strftime('%Y%m').astype(int)

# Convert MRL to numeric, replacing non-numeric values with NaN
df_all['MRL'] = pd.to_numeric(df_all['MRL'], errors='coerce')

# Convert AnalyticalResultValue to numeric, replacing non-numeric values with NaN and then filling NaNs with 0
df_all['AnalyticalResultValue'] = pd.to_numeric(df_all['AnalyticalResultValue'], errors='coerce').fillna(0)

# Add key field to df_all using 'PWSID', 'FacilityID', 'SamplePointID', 'SampleEventCode' with "_" separator
df_all['key'] = df_all[['PWSID', 'FacilityID', 'SamplePointID', 'SampleEventCode']].apply(lambda x: '_'.join(x), axis=1)

### Drop unnecessary fields

These fields are identifiers, duplicatative information, empty, or have one-value.

In [4]:
# Drop PWSName, Size, FacilityName, SamplePointName, SamplePointType, AssociatedFacilityID 
# AssociatedSamplePointID, SampleID, MonitoringRequirement, UCMR1SampleType
df_all.drop(columns=['PWSName', 'Size', 'FacilityName', 'SamplePointName', 'SamplePointType', 
                     'AssociatedFacilityID', 'AssociatedSamplePointID', 'SampleID', 'MonitoringRequirement', 
                     'UCMR1SampleType'], inplace=True)

### Create records for each key, contaminant, and collectiondate_yyyymm, ensuring the existing records aren't altered. Only missing records are added by forward and backward checking 

Run-time for this section is ~30 min.

In [5]:
import pandas as pd

def fill_missing_records(df_all, start_date='202301', end_date='202406'):
    """
    Fill in missing monthly records using a date spine while respecting unique
    contaminants per key. Optimized version with duplicate handling.
    
    Parameters:
    -----------
    df_all : pandas.DataFrame
        Input dataframe containing key, Contaminant, and date columns
    start_date : str
        Start date in YYYYMM format
    end_date : str
        End date in YYYYMM format
        
    Returns:
    --------
    pandas.DataFrame
        Filled dataframe with complete monthly records and is_new flag
    """
    # Create date range
    date_range = pd.date_range(
        start=pd.to_datetime(start_date, format='%Y%m'),
        end=pd.to_datetime(end_date, format='%Y%m'),
        freq='MS'
    )
    
    # Deduplicate records
    df_deduped = (df_all.sort_values('CollectionDate')
                       .drop_duplicates(['key', 'Contaminant', 'CollectionDate_FOM'], 
                                      keep='first')
                       .copy())
    
    # Store original records
    original_records = df_deduped[['key', 'Contaminant', 'CollectionDate_FOM']].copy()
    
    # Get unique key-contaminant pairs
    key_contam_pairs = df_deduped[['key', 'Contaminant']].drop_duplicates()
    
    # Create cartesian product of key-contaminant pairs and dates
    all_dates = pd.DataFrame({'CollectionDate_FOM': date_range})
    key_contam_dates = (key_contam_pairs.assign(key_dummy=1)
                                      .merge(all_dates.assign(key_dummy=1), on='key_dummy')
                                      .drop('key_dummy', axis=1))
    
    # Merge with original data to get all combinations
    df_filled = (key_contam_dates.merge(df_deduped, 
                                      on=['key', 'Contaminant', 'CollectionDate_FOM'],
                                      how='left'))
    
    # Forward fill and backward fill within groups
    def fill_group(group):
        # Select only the columns we want to fill
        fill_cols = group.columns.difference(['key', 'Contaminant', 'CollectionDate_FOM'])
        result = group.copy()
        # Explicitly call infer_objects() to handle the downcasting warning
        result[fill_cols] = (result[fill_cols]
                            .ffill()
                            .bfill()
                            .infer_objects(copy=False))
        return result

    # Modified groupby operation to avoid column conflicts
    df_filled = (df_filled.set_index(['key', 'Contaminant', 'CollectionDate_FOM'])
                         .groupby(level=['key', 'Contaminant'], group_keys=False)
                         .apply(fill_group))
    
    # Reset index without creating duplicate columns
    df_filled = df_filled.reset_index()
    
    # Mark new records using merge
    df_filled['is_new'] = ~df_filled[['key', 'Contaminant', 'CollectionDate_FOM']].apply(
        tuple, axis=1).isin(original_records[['key', 'Contaminant', 'CollectionDate_FOM']].apply(
        tuple, axis=1))
    
    # Update date columns
    df_filled['CollectionDate'] = df_filled['CollectionDate_FOM']
    df_filled['CollectionDate_YYYYMM'] = df_filled['CollectionDate'].dt.strftime('%Y%m').astype(int)
    
    print(f"Original records: {len(df_all)}")
    print(f"Records after deduplication: {len(df_deduped)}")
    print(f"Records after filling: {len(df_filled)}")
    print(f"New records added: {df_filled['is_new'].sum()}")
    print(f"Original records in output: {(~df_filled['is_new']).sum()}")
    
    return df_filled

# Run the function
filled_df = fill_missing_records(df_all)

Original records: 915941
Records after deduplication: 915722
Records after filling: 16461216
New records added: 15545494
Original records in output: 915722


### Checkpoint: Save or Load Progress

In [3]:
import pandas as pd
import os

# Define the directory path
save_dir = '/Users/kb/Library/CloudStorage/OneDrive-Personal/projects/cs_6463_final/modeling'

# Make sure the directory exists
# os.makedirs(save_dir, exist_ok=True)

# Save the DataFrames as pickle files
# filled_df.to_pickle(os.path.join(save_dir, 'filled_df.pkl'))
# df_all.to_pickle(os.path.join(save_dir, 'df_all.pkl'))

# Load pkl files
filled_df = pd.read_pickle(os.path.join(save_dir, 'filled_df.pkl'))
# df_all = pd.read_pickle(os.path.join(save_dir, 'df_all.pkl'))

### Add prior observation (t-1) of AnalyticalResultValue, month, and season as predictors

In [4]:
# Sort by key, CollectionDate_YYYYMM, and Contaminant
filled_df = filled_df.sort_values(['key', 'CollectionDate_YYYYMM', 'Contaminant'])

# Create shifted variable grouped by the specified columns
# Fill NA with the first value of each group (backfill)
filled_df['AnalyticalResultValue_prior'] = (filled_df.groupby(['key', 'Contaminant'])['AnalyticalResultValue']
                                          .shift(1))

# Fill remaining NAs with the first value in each group
filled_df['AnalyticalResultValue_prior'] = (filled_df.groupby(['key', 'Contaminant'])
                                          ['AnalyticalResultValue_prior']
                                          .transform(lambda x: x.fillna(x.iloc[0] if len(x) > 0 else 0)))

# Add month feature (1-12)
filled_df['month'] = filled_df['CollectionDate_YYYYMM'] % 100

# Add season
filled_df['season'] = pd.cut(filled_df['month'], 
                               bins=[0, 3, 6, 9, 12], 
                               labels=['Winter', 'Spring', 'Summer', 'Fall'], 
                               include_lowest=True)

### Pivot Additional Data Elements

In [5]:
# Create a pivot table to get all possible columns
all_columns_pivot = df_addtl.pivot_table(
    index=['PWSID', 'FacilityID', 'SamplePointID', 'SampleEventCode'],
    columns='AdditionalDataElement',
    values='Response',
    aggfunc=lambda x: list(x) if len(x) > 1 else x.iloc[0]
).reset_index()

# Find columns containing lists using map instead of applymap
list_columns = all_columns_pivot.map(lambda x: isinstance(x, list)).any()
columns_to_explode = list_columns[list_columns].index.tolist()

result_df = all_columns_pivot
for col in columns_to_explode:
    result_df = result_df.explode(col)

### Merge all data, zip code data, and additional data elements

In [6]:
# Merge the main data with additional elements
final_df = filled_df.merge(
    result_df,
    on=['PWSID', 'FacilityID', 'SamplePointID', 'SampleEventCode'],
    how='left'
)

# Merge with ZIP codes
# merged_df = final_df.merge(
#     df_zip,
#     on='PWSID',
#     how='left'
# )

# Quick validation checks
# print(f"Original rows in df_all: {len(df_all)}")
# print(f"Final merged rows: {len(final_df)}")
# print(f"Number of unique PWSIDs: {final_df['PWSID'].nunique()}")

### Checkpoint: Save or Load Progress

In [7]:
import pandas as pd
import os

# Define the directory path
save_dir = '/Users/kb/Library/CloudStorage/OneDrive-Personal/projects/cs_6463_final/modeling'

# Make sure the directory exists
# os.makedirs(save_dir, exist_ok=True)

# Save the DataFrames as pickle files
final_df.to_pickle(os.path.join(save_dir, 'final_df.pkl'))

# Load pkl files
# final_df = pd.read_pickle(os.path.join(save_dir, 'final_df.pkl'))

### Apply one-hot encoding and remove unnecessary fields

In [7]:
# Drop 'PWSID', 'FacilityID', 'SamplePointID', 'SampleEventCode', CollectionDate_FOM, CollectionDate, MRL, Units
# AnalyticalResultsSign, State, is_new
final_df.drop(columns=['PWSID', 'FacilityID', 'SamplePointID', 'SampleEventCode', 'CollectionDate_FOM', 
                       'CollectionDate', 'MRL', 'Units', 'AnalyticalResultsSign', 'State', 'is_new'], 
               inplace=True)


# One-hot encode Contaminant, FacilityWaterType, MethodID, Region, CollectionDate_YYYYMM, DisinfectantType
# LithiumOccurrence, LithiumTreatment, PFASOccurrence, PFASTreatment, PotentialPFASSources, PotentialPFASSourcesDetail
# TreatmentInformation
final_df_dmy = pd.get_dummies(final_df, columns=['Contaminant', 'FacilityWaterType', 'MethodID', 'Region', 
                                             'DisinfectantType', 'LithiumOccurrence', 'LithiumTreatment', 'PFASOccurrence', 
                                             'PFASTreatment', 'PotentialPFASSources', 'PotentialPFASSourcesDetail', 
                                             'TreatmentInformation'])

### Split data into training and testing datasets.

Since the task is to predict PFAS levels for 2024, use 2023 data for training and 2024 for testing.

In [8]:
# Create training dataset (2023 data)
tr_df = final_df_dmy[final_df_dmy['CollectionDate_YYYYMM'] // 100 == 2023].copy()
tr_df = tr_df.dropna() 
tr_y = tr_df['AnalyticalResultValue']
tr_x = tr_df.drop('AnalyticalResultValue', axis=1)

# Create test dataset (2024 data)
te_df = final_df_dmy[final_df_dmy['CollectionDate_YYYYMM'] // 100 == 2024].copy()
te_df = te_df.dropna() 
te_y = te_df['AnalyticalResultValue']
te_x = te_df.drop('AnalyticalResultValue', axis=1)

# Drop CollectionDate_YYYYMM from both sets
tr_x = tr_x.drop('CollectionDate_YYYYMM', axis=1)
te_x = te_x.drop('CollectionDate_YYYYMM', axis=1)

### XGBoost model with most important predictors and optimal hyperparameters

Iterations of XGBoost model were separately performed to identify the optimal hyperparameters. To reduce run-time in the final version, the optimal hyperparameter values are fixed.

The feature importance calculation is based on XGBoost's built-in feature importance mechanism, accessed through the feature_importances_ attribute. This attribute provides a numerical score for each feature that represents its contribution to model predictions.

The feature selection process is implemented in the _select_features method and consists of the following steps:

1. Initial Scoring   
*Creates a DataFrame containing features and their importance scores   
*Sorts features by importance in descending order

2. Feature Filtering   
*Implements a dual-criteria selection system:
    *Region-based features: Automatically included regardless of importance    
    *Non-region features: Selected based on importance threshold

3. Threshold Application   
*Uses the feature_threshold parameter (default: 0.02)   
*Only non-region features exceeding this threshold are retained

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from scipy.stats import pearsonr

def evaluate_and_display_metrics(y_true, y_pred, dataset_name=""):
    """Evaluate and display detailed metrics for model performance"""
    metrics = {
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R2': r2_score(y_true, y_pred),
        'Pearson_R': pearsonr(y_true, y_pred)[0]
    }
    
    print(f"\n{dataset_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric:<10} {value:e}")
    
    return metrics

class PFASPredictor:
    def __init__(self, feature_threshold=0.02):
        self.feature_threshold = feature_threshold
        self.model = None
        self.important_features = None
        self.label_encoders = {}
        self.feature_combinations = None
        
        # Best parameters from previous optimization
        self.best_params = {
            'objective': 'reg:squarederror',
            'n_estimators': 418,
            'max_depth': 4,
            'learning_rate': 0.018391464127050988,
            'min_child_weight': 3,
            'subsample': 0.827156877063853,
            'colsample_bytree': 0.9373610914155147,
            'gamma': 0.06669347293641514,
            'reg_alpha': 0.07382831113396796,
            'reg_lambda': 0.002998108064038156,
            'random_state': 42
        }
        
    def fit(self, tr_x, tr_y, te_x, te_y):
        """Train the model and store necessary information for future predictions"""
        # Store keys separately
        tr_keys = tr_x['key'].copy()
        te_keys = te_x['key'].copy()
        
        # Store state and date information
        self.states = sorted(list(set([k[:2] for k in tr_keys])))
        
        # Remove key from training features
        tr_x_model = tr_x.drop('key', axis=1)
        te_x_model = te_x.drop('key', axis=1)
        
        # Store the column names and their types
        self.feature_types = tr_x_model.dtypes
        
        # Store unique feature combinations
        self._store_feature_combinations(tr_x)
        
        # Preprocess the data and store label encoders
        tr_x_processed = self._preprocess_data(tr_x_model, fit=True)
        te_x_processed = self._preprocess_data(te_x_model, fit=False)
        
        # Log transform target variables
        tr_y_log = np.log1p(tr_y)
        te_y_log = np.log1p(te_y)
        
        # Create groups and compute weights
        groups = tr_x.apply(lambda row: f"{row['key']}_{self._get_contaminant(row)}", axis=1)
        weights = compute_sample_weight('balanced', tr_x_model.apply(self._get_contaminant, axis=1))
        
        # Train initial model and get feature importance
        initial_model = xgb.XGBRegressor(**self.best_params)
        initial_model.fit(tr_x_processed, tr_y_log, sample_weight=weights)
        
        # Select features
        self.important_features = self._select_features(initial_model, tr_x_processed.columns)
        
        # Train final model with selected features
        tr_x_selected = tr_x_processed[self.important_features]
        te_x_selected = te_x_processed[self.important_features]
        
        self.model = xgb.XGBRegressor(**self.best_params)
        print("\nTraining final model with selected features...")
        self.model.fit(
            tr_x_selected, tr_y_log,
            sample_weight=weights,
            eval_set=[(te_x_selected, te_y_log)],
            verbose=True
        )
        
        return self
    
    def evaluate_performance(self, tr_x, tr_y, te_x, te_y):
        """Evaluate model performance on training and test sets"""
        # Prepare data
        tr_x_processed = self._preprocess_data(tr_x.drop('key', axis=1), fit=False)
        te_x_processed = self._preprocess_data(te_x.drop('key', axis=1), fit=False)
        
        tr_x_selected = tr_x_processed[self.important_features]
        te_x_selected = te_x_processed[self.important_features]
        
        # Make predictions
        train_pred = np.expm1(self.model.predict(tr_x_selected))
        test_pred = np.expm1(self.model.predict(te_x_selected))
        
        # Evaluate and display metrics
        train_metrics = evaluate_and_display_metrics(tr_y, train_pred, "Training Set")
        test_metrics = evaluate_and_display_metrics(te_y, test_pred, "Test Set")
        
        # Print feature importance
        feature_importance = pd.DataFrame({
            'feature': self.important_features,
            'importance': self.model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\nFeature Importance:")
        print(feature_importance)
        
        return train_metrics, test_metrics
    
    def evaluate_by_contaminant(self, te_x, te_y):
        """Evaluate model performance by contaminant"""
        # Prepare test data
        te_x_processed = self._preprocess_data(te_x.drop('key', axis=1), fit=False)
        te_x_selected = te_x_processed[self.important_features]
        
        # Make predictions
        test_pred = np.expm1(self.model.predict(te_x_selected))
        
        # Get contaminant for each row
        contaminants = te_x.apply(self._get_contaminant, axis=1)
        
        # Create results DataFrame
        results = pd.DataFrame({
            'contaminant': contaminants,
            'true_values': te_y,
            'predictions': test_pred
        })
        
        # Evaluate by contaminant
        contam_results = []
        for contam, group in results.groupby('contaminant'):
            metrics = evaluate_and_display_metrics(
                group['true_values'], 
                group['predictions'],
                f"Contaminant: {contam}"
            )
            contam_results.append({
                'contaminant': contam,
                'count': len(group),
                'RMSE': metrics['RMSE'],
                'R2': metrics['R2']
            })
        
        contam_summary = pd.DataFrame(contam_results).set_index('contaminant').sort_values('RMSE')
        print("\nTest Set Performance by Contaminant:")
        print(contam_summary)
        
        return contam_summary
    
    def _preprocess_data(self, df, fit=False):
        """Preprocess data and handle label encoding"""
        df_processed = df.copy()
        
        for col in df_processed.columns:
            if df_processed[col].dtype.name == 'category':
                if fit:
                    self.label_encoders[col] = LabelEncoder()
                    df_processed[col] = self.label_encoders[col].fit_transform(df_processed[col])
                else:
                    df_processed[col] = self.label_encoders[col].transform(df_processed[col])
                    
        return df_processed
    
    def _store_feature_combinations(self, X):
        """Store unique feature combinations efficiently"""
        # Extract features excluding key
        features_df = X.drop('key', axis=1)
        
        # Get state from key
        features_df['state'] = X['key'].str[:2]
        
        # Store unique combinations as tuples
        self.feature_combinations = features_df.drop_duplicates().to_dict('records')
    
    def _get_contaminant(self, row):
        """Extract contaminant from feature columns"""
        contam_cols = [col for col in row.index if 'Contaminant_' in col]
        for col in contam_cols:
            if row[col] == 1:
                return col.replace('Contaminant_', '')
        return None
    
    def _select_features(self, model, columns):
        """Select important features while keeping all Region predictors"""
        feature_importance = pd.DataFrame({
            'feature': columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        # Select all Region features and only other important features
        region_features = [col for col in columns if col.startswith('Region_')]
        important_non_region = feature_importance[
            (~feature_importance['feature'].isin(region_features)) & 
            (feature_importance['importance'] > self.feature_threshold)
        ]['feature'].tolist()
        
        return region_features + important_non_region
    
    def generate_future_data(self, start_yearmonth=202407, end_yearmonth=202412):
        """Generate future prediction data for each state"""
        # Convert yearmonth to string format
        start_yearmonth = str(start_yearmonth)
        end_yearmonth = str(end_yearmonth)
        
        # Generate all month combinations
        years = range(int(start_yearmonth[:4]), int(end_yearmonth[:4]) + 1)
        months = range(1, 13)
        yearmonths = [f"{year}{month:02d}" for year in years 
                     for month in months 
                     if f"{year}{month:02d}" >= start_yearmonth 
                     and f"{year}{month:02d}" <= end_yearmonth]
        
        # Generate future rows more efficiently
        future_rows = []
        for combo in self.feature_combinations:
            state = combo.pop('state')  # Remove state from combo
            for ym in yearmonths:
                year = ym[:4]
                month = ym[4:6]
                key = f"{state}{year[2:]}{month}"
                
                row = combo.copy()
                row['key'] = key
                future_rows.append(row)
        
        future_data = pd.DataFrame(future_rows)
        return future_data
    
    def predict_future(self, start_yearmonth=202407, end_yearmonth=202412):
        """Generate and make predictions for future months"""
        # Generate future data
        future_data = self.generate_future_data(start_yearmonth, end_yearmonth)
        
        # Preprocess future data
        future_processed = self._preprocess_data(future_data.drop('key', axis=1), fit=False)
        future_processed_selected = future_processed[self.important_features]
        
        # Make predictions
        predictions = np.expm1(self.model.predict(future_processed_selected))
        
        # Create results DataFrame
        results = pd.DataFrame({
            'key': future_data['key'],
            'predicted_value': predictions
        })
        
        # Add state and date information
        results['state'] = results['key'].str[:2]
        results['year'] = '20' + results['key'].str[2:4]
        results['month'] = results['key'].str[4:6]
        results['yearmonth'] = results['year'] + results['month']
        
        return results.sort_values(['state', 'yearmonth'])

In [ ]:
predictor = PFASPredictor(feature_threshold=0.02)
predictor.fit(tr_x, tr_y, te_x, te_y)

# Get detailed evaluation metrics
train_metrics, test_metrics = predictor.evaluate_performance(tr_x, tr_y, te_x, te_y)

# Get performance by contaminant
contam_metrics = predictor.evaluate_by_contaminant(te_x, te_y)


Training final model with selected features...
[0]	validation_0-rmse:0.00222
[1]	validation_0-rmse:0.00219
[2]	validation_0-rmse:0.00215
[3]	validation_0-rmse:0.00215
[4]	validation_0-rmse:0.00211
[5]	validation_0-rmse:0.00208
[6]	validation_0-rmse:0.00204
[7]	validation_0-rmse:0.00201
[8]	validation_0-rmse:0.00197
[9]	validation_0-rmse:0.00194
[10]	validation_0-rmse:0.00191
[11]	validation_0-rmse:0.00188
[12]	validation_0-rmse:0.00185
[13]	validation_0-rmse:0.00185
[14]	validation_0-rmse:0.00182
[15]	validation_0-rmse:0.00179
[16]	validation_0-rmse:0.00176
[17]	validation_0-rmse:0.00173
[18]	validation_0-rmse:0.00170
[19]	validation_0-rmse:0.00167
[20]	validation_0-rmse:0.00165
[21]	validation_0-rmse:0.00162
[22]	validation_0-rmse:0.00159
[23]	validation_0-rmse:0.00157
[24]	validation_0-rmse:0.00154
[25]	validation_0-rmse:0.00152
[26]	validation_0-rmse:0.00150
[27]	validation_0-rmse:0.00147
[28]	validation_0-rmse:0.00145
[29]	validation_0-rmse:0.00145
[30]	validation_0-rmse:0.00143
[

/var/folders/5c/x0jsj1yd2336l637zvpwcy4h0000gn/T/ipykernel_97467/2317043947.py:17: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  'Pearson_R': pearsonr(y_true, y_pred)[0]
/var/folders/5c/x0jsj1yd2336l637zvpwcy4h0000gn/T/ipykernel_97467/2317043947.py:17: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  'Pearson_R': pearsonr(y_true, y_pred)[0]
/var/folders/5c/x0jsj1yd2336l637zvpwcy4h0000gn/T/ipykernel_97467/2317043947.py:17: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  'Pearson_R': pearsonr(y_true, y_pred)[0]



Contaminant: 11Cl-PF3OUdS Metrics:
MSE        1.691019e-11
RMSE       4.112200e-06
MAE        3.956338e-06
R2         0.000000e+00
Pearson_R  nan

Contaminant: 4:2 FTS Metrics:
MSE        3.150714e-11
RMSE       5.613122e-06
MAE        4.004559e-06
R2         9.945241e-01
Pearson_R  9.998568e-01

Contaminant: 6:2 FTS Metrics:
MSE        2.683131e-06
RMSE       1.638026e-03
MAE        2.559282e-05
R2         8.381005e-01
Pearson_R  9.171286e-01

Contaminant: 8:2 FTS Metrics:
MSE        2.977700e-11
RMSE       5.456831e-06
MAE        3.993574e-06
R2         9.946904e-01
Pearson_R  9.998714e-01

Contaminant: 9Cl-PF3ONS Metrics:
MSE        1.767716e-11
RMSE       4.204422e-06
MAE        3.961295e-06
R2         9.279770e-01
Pearson_R  9.971286e-01

Contaminant: ADONA Metrics:
MSE        5.259047e-11
RMSE       7.251929e-06
MAE        3.970151e-06
R2         6.449801e-01
Pearson_R  8.846130e-01

Contaminant: HFPO-DA Metrics:
MSE        8.850053e-10
RMSE       2.974904e-05
MAE        4.67709

/var/folders/5c/x0jsj1yd2336l637zvpwcy4h0000gn/T/ipykernel_97467/2317043947.py:17: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  'Pearson_R': pearsonr(y_true, y_pred)[0]
/var/folders/5c/x0jsj1yd2336l637zvpwcy4h0000gn/T/ipykernel_97467/2317043947.py:17: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  'Pearson_R': pearsonr(y_true, y_pred)[0]


### Checkpoint: Save or Load Progress

In [ ]:
import pickle

# After training and evaluation
save_dir = '/Users/kb/Library/CloudStorage/OneDrive-Personal/projects/cs_6463_final/modeling'

# Save the predictor object
with open(os.path.join(save_dir, 'pfas_predictor.pkl'), 'wb') as f:
    pickle.dump(predictor, f)

# Save the metrics
metrics_dict = {
    'train_metrics': train_metrics,
    'test_metrics': test_metrics,
    'contam_metrics': contam_metrics
}
with open(os.path.join(save_dir, 'metrics.pkl'), 'wb') as f:
    pickle.dump(metrics_dict, f)

# Load the predictor
# with open(os.path.join(save_dir, 'pfas_predictor.pkl'), 'rb') as f:
#     predictor = pickle.load(f)

# Load the metrics
# with open(os.path.join(save_dir, 'metrics.pkl'), 'rb') as f:
#     metrics_dict = pickle.load(f)

### Predict remainder of 2024

In [ ]:
import os
import pickle
import gc

# Clear memory
gc.collect()

# Generate predictions immediately
future_predictions = predictor.predict_future(202407, 202412)

### Checkpoint: Save or Load Progress

In [11]:
import os
import pickle
import gc

# Load the saved predictor and metrics
save_dir = '/Users/kb/Library/CloudStorage/OneDrive-Personal/projects/cs_6463_final/modeling'

# Save the future predictions
# with open(os.path.join(save_dir, 'future_predictions.pkl'), 'wb') as f:
#     pickle.dump(future_predictions, f)

# Load the future predictions
with open(os.path.join(save_dir, 'future_predictions.pkl'), 'rb') as f:
    future_predictions = pickle.load(f)

### Check Regional Results

In [ ]:
def analyze_future_predictions_by_region(future_predictions):
    """
    Analyze the distribution of predicted PFAS levels by region
    
    Parameters:
    future_predictions: DataFrame containing the predictions
    """
    # Add descriptive statistics for each region
    regional_stats = (future_predictions
        .groupby('state')
        .agg({
            'predicted_value': ['count', 'mean', 'std', 'min', 'max', 'median'],
        })
        .round(6)
    )
    
    # Flatten column names
    regional_stats.columns = ['count', 'mean', 'std', 'min', 'max', 'median']
    
    # Sort by mean predicted value
    regional_stats_sorted = regional_stats.sort_values('mean', ascending=False)
    
    print("\nRegional Prediction Summary:")
    print(regional_stats_sorted)
    
    # Monthly trends by region
    monthly_trends = (future_predictions
        .groupby(['state', 'yearmonth'])
        .agg({
            'predicted_value': 'mean'
        })
        .round(6)
        .reset_index()
        .pivot(index='state', columns='yearmonth', values='predicted_value')
    )
    
    print("\nMonthly Trends by Region:")
    print(monthly_trends)
    
    return regional_stats_sorted, monthly_trends

# Optional: Create a more detailed summary of predictions
def get_detailed_regional_summary(future_predictions):
    """Get detailed statistics about predictions by region and month"""
    summary = []
    
    for state in future_predictions['state'].unique():
        state_data = future_predictions[future_predictions['state'] == state]
        
        # Calculate trend
        monthly_means = state_data.groupby('yearmonth')['predicted_value'].mean()
        trend = 'Increasing' if monthly_means.iloc[-1] > monthly_means.iloc[0] else 'Decreasing'
        
        # Calculate volatility (standard deviation of month-to-month changes)
        volatility = monthly_means.pct_change().std()
        
        summary.append({
            'state': state,
            'total_predictions': len(state_data),
            'mean_value': state_data['predicted_value'].mean(),
            'trend': trend,
            'volatility': volatility,
            'highest_month': monthly_means.idxmax(),
            'lowest_month': monthly_means.idxmin(),
        })
    
    summary_df = pd.DataFrame(summary).sort_values('mean_value', ascending=False)
    print("\nDetailed Regional Prediction Summary:")
    print(summary_df)
    
    return summary_df

# Execute the analysis
regional_stats, monthly_trends = analyze_future_predictions_by_region(future_predictions)

# Get detailed summary
detailed_summary = get_detailed_regional_summary(future_predictions)

# Export to csv
detailed_summary.to_csv(os.path.join(save_dir, 'detailed_summary.csv'), index=False)
regional_stats.to_csv(os.path.join(save_dir, 'regional_stats.csv'))
monthly_trends.to_csv(os.path.join(save_dir, 'monthly_trends.csv'))


Regional Prediction Summary:
        count      mean       std       min       max    median
state                                                          
GU      29238  0.006212  0.013225  0.000003  0.159319  0.003694
MP      30888  0.004748  0.018869  0.000003  0.143745  0.000003
MN     139524  0.004638  0.025404  0.000003  0.609702  0.000003
DE     128040  0.003363  0.014317  0.000003  0.149935  0.000003
01      11022  0.002501  0.004565  0.000003  0.018281  0.000003
...       ...       ...       ...       ...       ...       ...
05      19668  0.000008  0.000014  0.000003  0.000071  0.000003
02       3828  0.000007  0.000012  0.000003  0.000053  0.000003
10       9570  0.000007  0.000012  0.000003  0.000053  0.000003
08      28710  0.000007  0.000012  0.000003  0.000054  0.000003
VI       5742  0.000007  0.000012  0.000003  0.000053  0.000003

[65 rows x 6 columns]

Monthly Trends by Region:
yearmonth    202407    202408    202409    202410    202411    202412
state             